In [ ]:
import os
import glob
from skimage import io
import matplotlib.pyplot as plt

from scipy import ndimage
from skimage.segmentation import clear_border 
from scipy import ndimage as ndi
#from skimage.segmentation import watershed
#from skimage.feature import peak_local_max
from skimage.segmentation import expand_labels
from skimage.morphology import remove_small_objects
import numpy as np

#import imageio

In [ ]:
prediction_path = r'D:\Institute of Genetics and Cancer\Neurosphere assay\Assess_Segmentation_Data\UNet-020223'
output_path = r'D:\Institute of Genetics and Cancer\Neurosphere assay\Assess_Segmentation_Data\UNet_30'

all_images =[]
for file in glob.glob(prediction_path + "\*.tif"):
    all_images.append(os.path.basename(file))

print(all_images)

In [ ]:
# Now we want to separate the two objects in image
# Generate the markers as local maxima of the distance to the background

#def watershed_segmentation(image):
#    distance = ndi.distance_transform_edt(image)
#    coords = peak_local_max(distance, footprint=np.ones((47, 47)), labels=image)
#    mask = np.zeros(distance.shape, dtype=bool)
#    mask[tuple(coords.T)] = True
#    markers, _ = ndi.label(mask)
#    labels = watershed(-distance, markers, mask=image)
#    return labels

In [ ]:
for image_name in all_images:
    # Load predictions
    img_filename = os.path.join(prediction_path, image_name)
    prediction_multlabel = io.imread(img_filename)

    # create binary image, fill holes and exclude edges
    prediction_binary = prediction_multlabel == 2 # create binary mask of class 2 (spheroid) pixels
    prediction_filled = ndimage.binary_fill_holes(prediction_binary) # fill holes
    prediction_cleared = clear_border(prediction_filled)# exclude objects on image edges

    # watershed based on distance transform
    #prediction_labels = watershed_segmentation(prediction_cleared)

    # expand labels 2 pixels to compensate for the pre-processing
    prediction_labels, _ = ndi.label(prediction_cleared) # convert into a label image
    prediction_filtered = remove_small_objects(prediction_labels, 10) # filter out small objects
    prediction_expanded = expand_labels(label_image=prediction_filtered, distance=2) # expand objects 2 pixels

    # export labels
    #imageio.imwrite(os.path.join(output_path, image_name), prediction_expanded)
    io.imsave(os.path.join(output_path, image_name), prediction_expanded)
